In [2]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TemporalAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, x):  # x: [B, T, H]
        weights = torch.softmax(self.attn(x), dim=1)  # [B, T, 1]
        context = torch.sum(weights * x, dim=1)       # [B, H]
        return context, weights


class KalmanLSTM(nn.Module):
    def __init__(self, input_size=6, hidden_size=128, num_layers=2, output_len=5, dropout=0.3):
        super().__init__()
        self.output_len = output_len
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.attn = TemporalAttention(hidden_size)
        self.norm = nn.LayerNorm(hidden_size)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, output_len * 2)
        )

    def forward(self, x):  # x: [B, T, 6]
        lstm_out, _ = self.lstm(x)  # [B, T, H]
        context, _ = self.attn(lstm_out)  # [B, H]
        context = self.norm(context)
        pred = self.fc(context)  # [B, output_len * 2]
        return pred.view(-1, self.output_len, 2)


In [5]:
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

# --- Load trajectory data ---
with open("/workspace/hjs/python/lstm_train/car_trajectories.pkl", "rb") as f:
    data = pickle.load(f)

input_len, pred_len = 10, 5
X_seqs, Y_seqs = [], []

for track_id, points in data.items():
    coords = np.array([[p[1], p[2]] for p in points])
    if len(coords) < input_len + pred_len:
        continue
    for i in range(len(coords) - input_len - pred_len + 1):
        X_seqs.append(coords[i:i+input_len])
        Y_seqs.append(coords[i+input_len:i+input_len+pred_len])

X = np.array(X_seqs)
Y = np.array(Y_seqs)
def add_kinematics(X):
    """
    X: [B, T, 2] → return [B, T, 6] with velocity and acceleration
    """
    velocity = np.diff(X, axis=1, prepend=X[:, :1])  # [B, T, 2]
    acceleration = np.diff(velocity, axis=1, prepend=velocity[:, :1])  # [B, T, 2]
    return np.concatenate([X, velocity, acceleration], axis=-1)  # [B, T, 6]


X = add_kinematics(X)  # Now shape: [B, T, 6]
print(f"Prepared dataset: X={X.shape}, Y={Y.shape}")
# --- Train/Val split ---
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- DataLoaders ---
BATCH_SIZE = 64
train_ds = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
val_ds = TensorDataset(torch.Tensor(X_val), torch.Tensor(Y_val))
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

Prepared dataset: X=(609284, 10, 6), Y=(609284, 5, 2)
Using device: cuda


In [6]:
def compute_metrics(pred, target, dt=1.0):
    ade = torch.mean(torch.norm(pred - target, dim=-1))  # [B, N]
    fde = torch.norm(pred[:, -1] - target[:, -1], dim=-1).mean()
    mse = F.mse_loss(pred, target)
    mae = F.l1_loss(pred, target)
    physics_loss = compute_physics_loss(pred, dt)
    return ade.item(), fde.item(), mse.item(), mae.item(), physics_loss.item()
def compute_physics_loss(pred, dt=1.0):
    """
    pred: [B, N, 2]
    """
    velocity = (pred[:, 1:] - pred[:, :-1]) / dt  # [B, N-1, 2]
    vel_diff = velocity[:, 1:] - velocity[:, :-1]  # acceleration diff
    acc_loss = torch.mean(vel_diff**2)
    return acc_loss
def train_model(model, train_loader, val_loader=None, epochs=50, lr=1e-3, lambda_phy=0.1,
                device='cuda', weight_decay=1e-5, early_stop_patience=5):

    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                           factor=0.5, patience=2)

    best_val_loss = float('inf')
    patience_counter = 0
    best_model = None

    for epoch in range(epochs):
        model.train()
        total_loss, total_mse, total_phy = 0.0, 0.0, 0.0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for x_batch, y_batch in pbar:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            pred = model(x_batch)

            mse_loss = criterion(pred, y_batch)
            physics_loss = compute_physics_loss(pred)
            loss = mse_loss + lambda_phy * physics_loss

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_mse += mse_loss.item()
            total_phy += physics_loss.item()

            pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "MSE": f"{mse_loss.item():.4f}",
                "Phys": f"{physics_loss.item():.4f}"
            })

        avg_train_loss = total_loss / len(train_loader)
        avg_train_mse = total_mse / len(train_loader)
        avg_train_phy = total_phy / len(train_loader)

        # --- Validation phase ---
        if val_loader:
            model.eval()
            val_loss, val_mse_total, val_mae_total = 0.0, 0.0, 0.0
            val_phy_total, val_ade_total, val_fde_total = 0.0, 0.0, 0.0
            count = 0

            with torch.no_grad():
                for x_val, y_val in val_loader:
                    x_val, y_val = x_val.to(device), y_val.to(device)
                    pred = model(x_val)

                    mse = criterion(pred, y_val)
                    phy = compute_physics_loss(pred)
                    loss = mse + lambda_phy * phy
                    val_loss += loss.item()

                    ade, fde, mse_val, mae_val, phy_val = compute_metrics(pred, y_val)
                    val_ade_total += ade * len(x_val)
                    val_fde_total += fde * len(x_val)
                    val_mse_total += mse_val * len(x_val)
                    val_mae_total += mae_val * len(x_val)
                    val_phy_total += phy_val * len(x_val)
                    count += len(x_val)

            avg_val_loss = val_loss / len(val_loader)
            scheduler.step(avg_val_loss)

            avg_val_ade = val_ade_total / count
            avg_val_fde = val_fde_total / count
            avg_val_mse = val_mse_total / count
            avg_val_mae = val_mae_total / count
            avg_val_phy = val_phy_total / count

            print(f"✅ Epoch {epoch+1:02d}/{epochs} | "
                  f"Train Loss: {avg_train_loss:.4f} (MSE={avg_train_mse:.4f}, Phys={avg_train_phy:.4f}) | "
                  f"Val Loss: {avg_val_loss:.4f} | MSE: {avg_val_mse:.4f} | MAE: {avg_val_mae:.4f} | "
                  f"ADE: {avg_val_ade:.4f} | FDE: {avg_val_fde:.4f} | Phys: {avg_val_phy:.4f}")

            # Early stopping
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                best_model = model.state_dict()
            else:
                patience_counter += 1
                if patience_counter >= early_stop_patience:
                    print("🛑 Early stopping triggered.")
                    break
        else:
            print(f"✅ Epoch {epoch+1:02d}/{epochs} | Train Loss: {avg_train_loss:.4f} "
                  f"(MSE={avg_train_mse:.4f}, Phys={avg_train_phy:.6f})")

    if best_model:
        torch.save(best_model, "best_kalman_lstm.pt")
        print("📦 Best model saved to 'best_kalman_lstm.pt'")




In [ ]:

# model = KalmanLSTM(input_size=6, hidden_size=128, num_layers=2, output_len=5, dropout=0.3)
# train_model(model, train_dl, val_dl,
#             epochs=100, lr=1e-3, lambda_phy=0.1,
#             device=device, weight_decay=1e-5, early_stop_patience=7)

In [7]:
import optuna

def objective(trial):
    # Sample hyperparameters
    hidden_size = trial.suggest_categorical("hidden_size", [64, 128, 256])
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    lambda_phy = trial.suggest_float("lambda_phy", 0.01, 1.0)

    model = KalmanLSTM(input_size=6, hidden_size=hidden_size,
                       num_layers=num_layers, output_len=5, dropout=dropout)
    
    # Train for fewer epochs to keep tuning fast
    train_model(model, train_dl, val_dl,
                epochs=25, lr=lr, lambda_phy=lambda_phy,
                device=device, weight_decay=weight_decay, early_stop_patience=7)

    # Load best model for evaluation
    model.load_state_dict(torch.load("best_kalman_lstm.pt"))
    model.to(device)
    model.eval()

    # Evaluate on validation set
    val_mse = 0.0
    count = 0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for x_val, y_val in val_dl:
            x_val, y_val = x_val.to(device), y_val.to(device)
            pred = model(x_val)
            val_mse += criterion(pred, y_val).item() * len(x_val)
            count += len(x_val)

    return val_mse / count  # Minimize this

# Start optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("🏆 Best hyperparameters:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

print(f"Best Validation MSE: {study.best_value:.4f}")


[I 2025-06-24 08:04:56,655] A new study created in memory with name: no-name-94a5eb2a-8165-4963-a56f-96d38654db27


✅ Epoch 01/25 | Train Loss: 0.0634 (MSE=0.0603, Phys=0.0033) | Val Loss: 0.0212 | MSE: 0.0205 | MAE: 0.0882 | ADE: 0.1370 | FDE: 0.1416 | Phys: 0.0008


✅ Epoch 02/25 | Train Loss: 0.0212 (MSE=0.0201, Phys=0.0012) | Val Loss: 0.0223 | MSE: 0.0190 | MAE: 0.0788 | ADE: 0.1325 | FDE: 0.1396 | Phys: 0.0035


✅ Epoch 03/25 | Train Loss: 0.0193 (MSE=0.0185, Phys=0.0009) | Val Loss: 0.0294 | MSE: 0.0288 | MAE: 0.1098 | ADE: 0.1912 | FDE: 0.2188 | Phys: 0.0006


✅ Epoch 04/25 | Train Loss: 0.0184 (MSE=0.0178, Phys=0.0006) | Val Loss: 0.1929 | MSE: 0.1926 | MAE: 0.3013 | ADE: 0.5575 | FDE: 0.6599 | Phys: 0.0004
🛑 Early stopping triggered.
📦 Best model saved to 'best_kalman_lstm.pt'


...

✅ Epoch 08/25 | Train Loss: 0.0074 (MSE=0.0073, Phys=0.0001) | Val Loss: 0.0072 | MSE: 0.0071 | MAE: 0.0284 | ADE: 0.0495 | FDE: 0.0749 | Phys: 0.0001


✅ Epoch 09/25 | Train Loss: 0.0073 (MSE=0.0072, Phys=0.0001) | Val Loss: 0.0072 | MSE: 0.0071 | MAE: 0.0279 | ADE: 0.0489 | FDE: 0.0739 | Phys: 0.0001


✅ Epoch 10/25 | Train Loss: 0.0072 (MSE=0.0071, Phys=0.0001) | Val Loss: 0.0076 | MSE: 0.0073 | MAE: 0.0297 | ADE: 0.0523 | FDE: 0.0758 | Phys: 0.0003


✅ Epoch 11/25 | Train Loss: 0.0071 (MSE=0.0070, Phys=0.0001) | Val Loss: 0.0084 | MSE: 0.0084 | MAE: 0.0347 | ADE: 0.0619 | FDE: 0.0868 | Phys: 0.0000


Epoch 12/25:  94%|█████████▎| 7139/7617 [00:19<00:01, 361.13it/s, Loss=0.0027, MSE=0.0026, Phys=0.0001]

In [ ]:
import pandas as pd

def summarize_study(study):
    rows = []
    for t in study.trials:
        row = {
            "trial": t.number,
            "state": t.state.name,
            "value (Val MSE)": t.value,
            "hidden_size": t.params.get("hidden_size"),
            "num_layers": t.params.get("num_layers"),
            "dropout": round(t.params.get("dropout", 0), 3),
            "lr": f"{t.params.get('lr'):.1e}",
            "weight_decay": f"{t.params.get('weight_decay'):.1e}",
            "lambda_phy": round(t.params.get("lambda_phy", 0), 3)
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    df = df.sort_values("value (Val MSE)").reset_index(drop=True)

    print("\n📋 Trial Summary (sorted by best Val MSE):")
    print(df.head(10).to_string(index=False))  # Show top 10 results

    # Save to CSV
#     df.to_csv("optuna_trial_summary.csv", index=False)
#     print("\n📁 Full trial summary saved to 'optuna_trial_summary.csv'.")

summarize_study(study)
